# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat

In [3]:
# set the parameters to run the script
# directories = [r'E:\tumor_new_measurement']
directories = [r'C:\Users\romai\Documents\PathologyPaper\data\HealthyMethods']
calib_directory = r'C:\Users\romai\Documents\PathologyPaper\calib'

In [5]:
# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = True, parameter_set = 'TheoniPics', PDDN = False)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{}
{} 2022-07-06_T_autopsy-bloc_BF_FR_M1_1
./temp_processing\2022-07-06_T_autopsy-bloc_BF_FR_M1_1 2022-07-06 00:00:00
{}
{} 2022-07-07_T_autopsy-cryo-AF_FR_S2_1
./temp_processing\2022-07-07_T_autopsy-cryo-AF_FR_S2_1 2022-07-07 00:00:00


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:52<00:00, 52.67s/it]


['C:\\Users\\romai\\Documents\\PathologyPaper\\data\\HealthyMethods\\2022-07-06_T_autopsy-bloc_BF_FR_M1_1',
 'C:\\Users\\romai\\Documents\\PathologyPaper\\data\\HealthyMethods\\2022-07-07_T_autopsy-cryo-AF_FR_S2_1']